In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

In [9]:
# preparing data
# adding noise means that the network is less able to memorize training samples

X_numpy, y_numpy = datasets.make_classification(n_samples=100, n_features=10, n_classes=2, weights=[0.7,0.3], random_state=4)

In [10]:
X_numpy.shape, y_numpy.shape

((100, 10), (100,))

In [11]:
X_numpy[:5]

array([[ 0.05943278, -1.39305057,  1.39913499,  1.05991603,  0.99302185,
        -0.32392025,  0.01373755, -0.39449817,  1.09713425,  1.53002369],
       [-0.06954632, -0.98297097, -2.72879706, -0.34465271,  1.01355006,
        -0.24649635,  0.61412073,  0.04412067, -1.04948693,  0.45958178],
       [ 0.74805119,  1.14819764,  1.7783817 ,  0.95693258, -2.7879091 ,
        -0.21273364, -0.12775208,  0.45122724, -0.4264791 ,  0.76069577],
       [-1.28460768, -0.08098206, -1.38236082,  1.43110647,  0.73667157,
        -0.94354761,  0.90854036,  0.31885187,  0.79247004, -0.96990561],
       [ 0.29419801,  0.35225773,  1.66799846, -0.08113937,  1.66972899,
         0.29945447, -0.48396097, -0.61140276, -0.83272344,  1.94530297]])

In [12]:
y_numpy[:5]

array([1, 0, 1, 0, 1])

In [13]:
values, counts = np.unique(y_numpy, return_counts=True)
values, counts

(array([0, 1]), array([70, 30]))

In [14]:
# converting to tensor

X = torch.from_numpy(X_numpy.astype(np.float32))
y = torch.from_numpy(y_numpy.astype(np.float32))

In [15]:
X.shape, y.shape

(torch.Size([100, 10]), torch.Size([100]))

In [16]:
y.shape[0]

100

In [17]:
# reshaping y

y = y.view(y.shape[0], 1)
y.shape

torch.Size([100, 1])

In [18]:
n_samples, n_features = X.shape
n_samples, n_features

(100, 10)

In [19]:
input_size = n_features
output_size = 1

In [20]:
# linear model f = wx + b, sigmoid at end

class LogisticRegression(nn.Module):
    def __init__(self, n_input_features, n_output_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, n_output_features)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = LogisticRegression(input_size, output_size)

In [21]:
# loss and optimizer

learning_rate = 0.01

loss_func = nn.BCELoss()                # binary cross entropy
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [22]:
# training loop

num_epochs = 200
for epoch in range(num_epochs):

    # forward pass and loss
    y_hat = model(X)
    loss = loss_func(y_hat, y)                  # loss_func is an object used as a function
    
    # backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()                       # dy_hat/dw = 0 after every epoch

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 10, loss = 0.6340
epoch: 20, loss = 0.6057
epoch: 30, loss = 0.5800
epoch: 40, loss = 0.5566
epoch: 50, loss = 0.5354
epoch: 60, loss = 0.5161
epoch: 70, loss = 0.4984
epoch: 80, loss = 0.4822
epoch: 90, loss = 0.4674
epoch: 100, loss = 0.4538
epoch: 110, loss = 0.4412
epoch: 120, loss = 0.4296
epoch: 130, loss = 0.4188
epoch: 140, loss = 0.4088
epoch: 150, loss = 0.3995
epoch: 160, loss = 0.3909
epoch: 170, loss = 0.3828
epoch: 180, loss = 0.3751
epoch: 190, loss = 0.3680
epoch: 200, loss = 0.3613


In [23]:
with torch.no_grad():
    y_predicted = model(X)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y).sum() / float(y.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.9000


In [24]:
y_predicted[:10]

tensor([[0.7490],
        [0.0794],
        [0.6481],
        [0.1446],
        [0.8042],
        [0.1816],
        [0.5647],
        [0.2000],
        [0.2437],
        [0.5848]])

In [25]:
y_predicted_cls[:10]

tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.]])

In [26]:
y[:10]

tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.]])